In [13]:
import pandas as pd
import os
import rapidjson
import numpy as np

In [14]:
node_path = "np_node-template.csv"
tag_path = "np_tag-template.csv"
os.chmod(node_path, 0o777)
colors = pd.read_csv('colors.csv', index_col='np_color_id')

In [15]:
def min_max_scaling(series):
    return (series - series.min()) / (series.max() - series.min())

In [16]:
geoinfo_df = pd.read_csv('GeoLocationInfo.csv')
node_df = pd.read_csv(node_path)
tag_df = pd.read_csv(tag_path)
data =  pd.read_csv("2d SDG Dataset.csv", header=0, low_memory=False).reset_index(drop=True)
indicators_with_index = ['GeoAreaCode', 'GeoAreaName', 'TimePeriod', '1.1.1', '1.4.1', '6.1.1', '6.2.1', '6.4.1', 
             '13.1.1','13.1.2','17.1.1', '17.1.2']
subset_columns = []

# Check for regular string columns
subset_columns += [col for col in data.columns if col in ['GeoAreaCode', 'GeoAreaName', 'TimePeriod']]

# Check for JSON columns
for col in data.columns:
    if col not in ['GeoAreaCode', 'TimePeriod']:
        try:
            col_json = rapidjson.loads(col)
            if col_json.get("Indicator") in indicators_with_index:
                subset_columns.append(col)
        except rapidjson.JSONDecodeError:
            continue
# subset_columns = [col for col in data.columns if any(indicator in col for indicator in indicators_with_index)]
subset = data[subset_columns]
# subset.drop(subset.columns[[9,10,11,12,13]], axis=1, inplace=True)

timeperiods = [2000, 2005, 2010, 2015, 2020]
# drop_cols = ['{"Indicator": "13.1.1", "SeriesCode": "VC_DSR_MMHN"}','{"Indicator": "13.1.1", "SeriesCode": "VC_DSR_MORT"}','{"Indicator": "13.1.1", "SeriesCode": "VC_DSR_PDAN"}',
#              '{"Indicator": "13.1.1", "SeriesCode": "VC_DSR_PDLN"}','{"Indicator": "13.1.1", "SeriesCode": "VC_DSR_PDYN"}','{"Indicator": "17.1.1", "SeriesCode": "GR_G14_XDC"}',
#              '{"Indicator": "13.1.1", "SeriesCode": "VC_DSR_AFFCT"}','{"Indicator": "13.1.1", "SeriesCode": "VC_DSR_DAFF"}', '{"Indicator": "13.1.1", "SeriesCode": "VC_DSR_IJILN"}',
#              '{"Indicator": "13.1.1", "SeriesCode": "VC_DSR_MISS"}','{"Indicator": "13.1.1", "SeriesCode": "VC_DSR_MTMP"}']
subset = subset[subset['TimePeriod'].isin(timeperiods)]
# subset = subset.drop(drop_cols, axis=1)

for country in subset.GeoAreaName.unique():
    print(f"Working on {country}")
    subset_to_scale = subset[subset.GeoAreaName == 	country].iloc[:, 3:]
    scaled_values = subset_to_scale.apply(min_max_scaling, axis=0)
    subset.loc[subset.GeoAreaName == country, subset_to_scale.columns] = scaled_values.values
    


valid_geo_area_names = geoinfo_df['GeoAreaName'].unique()
subset = subset[subset['GeoAreaName'].isin(valid_geo_area_names)]
mask = subset.apply(lambda row: any(isinstance(cell, str) or (isinstance(cell, (int, float)) and cell > 100) for cell in row), axis=1)
subset = subset[mask]
subset = subset.fillna(0)
subset.drop(subset.columns[[1]], axis=1, inplace=True)
subset.to_csv('subset.csv', index=False)

Working on Afghanistan
Working on Albania
Working on Algeria
Working on American Samoa
Working on Andorra
Working on Angola
Working on Antigua and Barbuda
Working on Azerbaijan
Working on Argentina
Working on Australia
Working on Austria
Working on Bahamas
Working on Bahrain
Working on Bangladesh
Working on Armenia
Working on Barbados
Working on Melanesia
Working on Belgium
Working on Micronesia
Working on Bermuda
Working on Polynesia
Working on Bhutan
Working on Bolivia
Working on Bosnia and Herzegovina
Working on Botswana
Working on Bouvet Island
Working on Brazil
Working on Belize
Working on Solomon Islands
Working on British Virgin Islands
Working on Brunei Darussalam
Working on Bulgaria
Working on Myanmar
Working on Burundi
Working on Belarus
Working on Cambodia
Working on Cameroon
Working on Canada
Working on Cabo Verde
Working on Cayman Islands
Working on Central African Republic
Working on Sri Lanka
Working on Chad
Working on Chile
Working on China
Working on Christmas Island
W

In [17]:
subset.describe()

,GeoAreaCode,TimePeriod,"{""Indicator"": ""1.1.1"", ""SeriesCode"": ""SI_POV_DAY1"", ""Age"": ""Adult"", ""Location"": ""ALLAREA"", ""Sex"": ""BOTHSEX""}","{""Indicator"": ""1.1.1"", ""SeriesCode"": ""SI_POV_DAY1"", ""Age"": ""Child"", ""Location"": ""ALLAREA"", ""Sex"": ""BOTHSEX""}","{""Indicator"": ""1.1.1"", ""SeriesCode"": ""SI_POV_DAY1"", ""Age"": ""Elderly"", ""Location"": ""ALLAREA"", ""Sex"": ""BOTHSEX""}","{""Indicator"": ""1.1.1"", ""SeriesCode"": ""SI_POV_DAY1"", ""Location"": ""ALLAREA"", ""Sex"": ""BOTHSEX""}","{""Indicator"": ""1.1.1"", ""SeriesCode"": ""SI_POV_DAY1"", ""Location"": ""ALLAREA"", ""Sex"": ""FEMALE""}","{""Indicator"": ""1.1.1"", ""SeriesCode"": ""SI_POV_DAY1"", ""Location"": ""ALLAREA"", ""Sex"": ""MALE""}","{""Indicator"": ""1.1.1"", ""SeriesCode"": ""SI_POV_DAY1"", ""Location"": ""RURAL"", ""Sex"": ""BOTHSEX""}","{""Indicator"": ""1.1.1"", ""SeriesCode"": ""SI_POV_DAY1"", ""Location"": ""URBAN"", ""Sex"": ""BOTHSEX""}",...,"{""Indicator"": ""6.2.1"", ""SeriesCode"": ""SH_SAN_HNDWSH"", ""Location"": ""ALLAREA""}","{""Indicator"": ""6.2.1"", ""SeriesCode"": ""SH_SAN_HNDWSH"", ""Location"": ""RURAL""}","{""Indicator"": ""6.2.1"", ""SeriesCode"": ""SH_SAN_HNDWSH"", ""Location"": ""URBAN""}","{""Indicator"": ""6.2.1"", ""SeriesCode"": ""SH_SAN_SAFE"", ""Location"": ""ALLAREA""}","{""Indicator"": ""6.2.1"", ""SeriesCode"": ""SH_SAN_SAFE"", ""Location"": ""RURAL""}","{""Indicator"": ""6.2.1"", ""SeriesCode"": ""SH_SAN_SAFE"", ""Location"": ""URBAN""}","{""Indicator"": ""6.4.1"", ""SeriesCode"": ""ER_H2O_WUEYST"", ""Activity"": ""TOTAL""}","{""Indicator"": ""6.4.1"", ""SeriesCode"": ""ER_H2O_WUEYST"", ""Activity"": ""INDUSTRIES""}","{""Indicator"": ""6.4.1"", ""SeriesCode"": ""ER_H2O_WUEYST"", ""Activity"": ""ISIC4_A01_A0210_A0322""}","{""Indicator"": ""6.4.1"", ""SeriesCode"": ""ER_H2O_WUEYST"", ""Activity"": ""ISIC4_GTT""}"
count,1209.000000,1209.000000,1209.000000,1209.000000,1209.000000,1209.000000,1209.000000,1209.000000,1209.000000,1209.000000,...,1209.000000,1209.000000,1209.000000,1209.000000,1209.000000,1209.000000,1209.000000,1209.000000,1209.000000,1209.000000
mean,435.384615,2009.991729,0.066612,0.058073,0.058357,0.098358,0.066092,0.065224,0.060301,0.063255,...,0.112905,0.067920,0.067754,0.254219,0.176021,0.223672,0.299794,0.302836,0.284328,0.343777
std,253.348921,7.071063,0.233319,0.218664,0.217904,0.273777,0.233252,0.230853,0.220563,0.226807,...,0.292099,0.224521,0.224197,0.363945,0.324352,0.354181,0.394623,0.386570,0.388160,0.395082
min,4.000000,2000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,214.000000,2005.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,438.000000,2010.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.042857,0.000000,0.152918
75%,654.000000,2015.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.499690,0.216684,0.441976,0.651569,0.622322,0.571429,0.732660
max,894.000000,2020.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [21]:
%%time
node_df = pd.read_csv(node_path)
tag_df = pd.read_csv(tag_path)
previous_geo_code = ""
previous_time_period = ""
previous_indicator = ""
previous_goal = ""
previous_series = ""
ring_location = -115
series_ring_location = 0
petal_location = -15
num_goals = 4
data_id=1
id_iter=1
ind_color_iter=7

for idx, row in subset.iterrows():
    rod_location = -180
    ind_color_iter = 7
    new_pin = node_df.iloc[0].copy()
    new_tag = tag_df.iloc[0].copy()
    
    for column, value in row.items():
        
        if column == 'GeoAreaCode':
            if row.GeoAreaCode == previous_geo_code:
                ring_location += 20
            else:
                ring_location = -115
                pin_id = id_iter
                id_iter+=1
                geoinfo = geoinfo_df[geoinfo_df.GeoAreaCode == row.GeoAreaCode]
                # new_pin[["np_node_id", "np_data_id",  "np_tag_id", "record_id"]] = int(pin_id)
                new_pin[["np_node_id", "np_tag_id", "record_id"]] = pin_id
                new_pin['np_table_id'] = 1
                new_pin['scale_x'] = .5
                new_pin['scale_y'] = .5
                new_pin['scale_z'] = .5
                new_pin['translate_x'] = geoinfo['translate_x'].values[0]
                new_pin['translate_y'] = geoinfo['translate_y'].values[0]
                new_pin['np_geometry_id'] = 19
                new_pin['np_topo_id'] = 6
                new_pin['np_color_id'] = 20
                new_pin[['color_r', 'color_g', 'color_b']] = [50, 101, 101]
                new_tag[["np_tag_id", "record_id"]] = pin_id
                new_tag["table_id"] = 1
                new_tag["title"] =  geoinfo_df[geoinfo_df.GeoAreaCode == row.GeoAreaCode].GeoAreaName.reset_index(drop=True).iloc[0]
                new_tag["description"] = 'Country'
                node_df = pd.concat([node_df, pd.DataFrame(new_pin).T], ignore_index=True)
                tag_df = pd.concat([tag_df, pd.DataFrame(new_tag).T], ignore_index=True)
                previous_geo_code = row.GeoAreaCode

        
        elif column == 'TimePeriod':
            ring_id = id_iter
            id_iter+=1
            # print(str(row.GeoAreaCode)+str(row.TimePeriod))
            new_year_ring = node_df.iloc[1].copy()
            new_year_tag = tag_df.iloc[0].copy()
            # new_year_ring[["np_node_id", "np_data_id",  "np_tag_id", "record_id"]] = int(ring_id)
            new_year_ring[["np_node_id", "np_tag_id", "record_id"]] = ring_id
            new_year_ring["parent_id"] = pin_id
            new_year_ring["branch_level"]=2
            new_year_ring["translate_x"] = ring_location
            new_year_ring[['scale_x', 'scale_y', 'scale_z']] = .5
            new_year_ring['np_table_id'] = 1
            new_year_ring['np_color_id'] = 20
            new_year_ring[['color_r', 'color_g', 'color_b']] = [55, 190, 190]
            new_year_tag[["np_tag_id", "record_id"]] = ring_id
            new_year_tag["table_id"] = 1
            new_year_tag["title"] = row.TimePeriod
            new_year_tag["description"] = "Year"
            node_df = pd.concat([node_df, pd.DataFrame(new_year_ring).T], ignore_index=True)
            tag_df = pd.concat([tag_df, pd.DataFrame(new_year_tag).T], ignore_index=True)
            previous_time_period = row.TimePeriod
        else:
            discriminator = rapidjson.loads(column)
            current_indicator = discriminator["Indicator"]
            current_series = discriminator["SeriesCode"]
            current_goal = current_indicator.split('.')[0]
            if current_indicator == previous_indicator:
                
                series_ring_location -= 5
                if current_series == previous_series:
                    petal_id=id_iter
                    id_iter += 1
                    petal_tag_title = ""
                    for disc_type, type_value in discriminator.items():
                        if disc_type not in ['Indicator', 'SeriesCode']:
                            petal_tag_title += type_value
                    new_subseries_petal = node_df.iloc[1].copy()
                    new_subseries_petal_tag = tag_df.iloc[0].copy()
                    new_subseries_petal[["np_node_id", "np_data_id",  "np_tag_id", "record_id"]] = int(petal_id)
                    # new_subseries_petal[["np_node_id", "np_tag_id", "record_id"]] = petal_id
                    new_subseries_petal["parent_id"] = series_id
                    new_subseries_petal["branch_level"]=5
                    new_subseries_petal["translate_x"] = petal_location
                    new_subseries_petal[['scale_x', 'scale_y', 'scale_z']] = 1
                    new_subseries_petal["np_geometry_id"] = 3
                    if value == 0 :
                        new_subseries_petal["np_geometry_id"] = 2
                        new_subseries_petal["np_topo_id"] = 10
                    else:
                        new_subseries_petal["np_geometry_id"] = 3
                        new_subseries_petal["np_topo_id"] = 2
                    new_subseries_petal['np_table_id'] = 1
                    new_subseries_petal[['scale_x','scale_y','scale_z']] = 1+.5*value
                    if 'Sex' in discriminator:
                        if discriminator['Sex'] == 'MALE':
                            new_subseries_petal['np_color_id'] = 3
                            new_subseries_petal[['color_r', 'color_g', 'color_b']] = colors.iloc[3]
                        elif discriminator['Sex'] == 'FEMALE':
                            new_subseries_petal['np_color_id'] = 2
                            new_subseries_petal[['color_r', 'color_g', 'color_b']] = colors.iloc[2]
                        else: 
                            new_subseries_petal['np_color_id'] = 4
                            new_subseries_petal[['color_r', 'color_g', 'color_b']] = colors.iloc[4]
                    new_subseries_petal_tag[["np_tag_id", "record_id"]] = petal_id
                    new_subseries_petal_tag["table_id"] = 1
                    new_subseries_petal_tag["title"] = petal_tag_title
                    new_subseries_petal_tag["description"] = "Series sub"
                    node_df = pd.concat([node_df, pd.DataFrame(new_subseries_petal).T], ignore_index=True)
                    tag_df = pd.concat([tag_df, pd.DataFrame(new_subseries_petal_tag).T], ignore_index=True)
                    petal_location += 30
                    previous_series = current_series
                else:
                    petal_location = -15
                    series_id=id_iter
                    id_iter += 1
                    petal_id=id_iter
                    id_iter += 1
                    new_series_ring = node_df.iloc[1].copy()
                    new_series_tag = tag_df.iloc[0].copy()
                    new_series_ring[["np_node_id", "np_data_id",  "np_tag_id", "record_id"]] = int(series_id)
                    new_series_ring[["np_node_id", "np_tag_id", "record_id"]] = series_id
                    new_series_ring["parent_id"] = indicator_id
                    new_series_ring["branch_level"]=4
                    new_series_ring["translate_x"] = series_ring_location
                    new_series_ring[['scale_x', 'scale_y', 'scale_z']] = 1
                    new_series_ring['np_table_id'] = 1
                    if value == 0:
                        new_series_ring["np_geometry_id"] = 6
                        new_series_ring["np_topo_id"] = 11
                    else:
                        new_series_ring["np_topo_id"] = 7
                        new_series_ring["np_topo_id"] = 3
                    new_series_ring[['scale_x','scale_y','scale_z']] = 1+.5*value
                    new_series_tag[["np_tag_id", "record_id"]] = series_id
                    new_series_tag["table_id"] = 1
                    new_series_tag["title"] = current_series
                    new_series_tag["description"] = "Series Code"
                    node_df = pd.concat([node_df, pd.DataFrame(new_series_ring).T], ignore_index=True)
                    tag_df = pd.concat([tag_df, pd.DataFrame(new_series_tag).T], ignore_index=True)
                    previous_series = current_series
            else:
                indicator_id=id_iter
                id_iter+=1
                series_ring_location = 0
                new_indicator_bar = node_df.iloc[1].copy()
                new_indicator_tag = tag_df.iloc[0].copy()
                new_indicator_bar[["np_node_id", "np_data_id",  "np_tag_id", "record_id"]] = int(indicator_id)
                # new_indicator_bar[["np_node_id", "np_tag_id", "record_id"]] = indicator_id
                new_indicator_bar["parent_id"] = ring_id
                new_indicator_bar["branch_level"]=3
                new_indicator_bar["translate_x"] = rod_location
                rod_location += 40
                new_indicator_bar["np_geometry_id"] = 19
                new_indicator_bar["np_topo_id"] = 6
                new_indicator_bar['np_table_id'] = 1
                if current_goal == previous_goal:
                    new_indicator_bar['np_color_id'] = ind_color_iter
                    new_indicator_bar[['color_r', 'color_g', 'color_b']] = colors.iloc[ind_color_iter]
                else:
                    ind_color_iter += 1
                    new_indicator_bar['np_color_id'] = ind_color_iter
                    new_indicator_bar[['color_r', 'color_g', 'color_b']] = colors.iloc[ind_color_iter]
                new_indicator_tag["table_id"] = 1
                new_indicator_tag[["np_tag_id", "record_id"]] = indicator_id
                new_indicator_tag["title"] = current_indicator
                new_indicator_tag["description"] = "Indicator"
                node_df = pd.concat([node_df, pd.DataFrame(new_indicator_bar).T], ignore_index=True)
                tag_df = pd.concat([tag_df, pd.DataFrame(new_indicator_tag).T], ignore_index=True)
                previous_indicator = current_indicator
                previous_goal = current_indicator.split('.')[0]
                series_id=id_iter
                id_iter += 1
                new_series_ring = node_df.iloc[1].copy()
                new_series_tag = tag_df.iloc[0].copy()
                new_series_ring[["np_node_id", "np_data_id",  "np_tag_id", "record_id"]] = int(series_id)
                # new_series_ring[["np_node_id", "np_tag_id", "record_id"]] = series_id
                new_series_ring["parent_id"] = indicator_id
                new_series_ring["branch_level"]=4
                new_series_ring["translate_x"] = series_ring_location
                new_series_ring[['scale_x', 'scale_y', 'scale_z']] = 1
                new_series_ring['np_table_id'] = 1
                if value == 0:
                    new_series_ring["np_geometry_id"] = 6
                    new_series_ring["np_topo_id"] = 11
                else:
                    new_series_ring["np_topo_id"] = 7
                    new_series_ring["np_topo_id"] = 3
                new_series_ring[['scale_x','scale_y','scale_z']] = 1+.5*value
                new_series_tag[["np_tag_id", "record_id"]] = series_id
                new_series_tag["table_id"] = 1
                new_series_tag["title"] = current_series
                new_series_tag["description"] = "Series Code"
                node_df = pd.concat([node_df, pd.DataFrame(new_series_ring).T], ignore_index=True)
                tag_df = pd.concat([tag_df, pd.DataFrame(new_series_tag).T], ignore_index=True)
                previous_series=current_series
                
node_df = node_df.iloc[2:]
tag_df = tag_df.iloc[1:]

CPU times: total: 4min 29s
Wall time: 9min 37s


In [22]:
node_ints = ['np_node_id','type','np_data_id','selected','parent_id','branch_level','child_id','np_tag_id','np_palette_id','np_ch_in_id','np_ch_out_id','ch_sync_time','np_palette_id_alt','np_color_id_alt',
            'np_material_id','np_geometry_id', 'np_color_id', 'color_fade','np_texture_id','hide','freeze','np_topo_id','subspace','trigger_hi_x','trigger_hi_y','trigger_hi_z','trigger_lo_x','trigger_lo_y','trigger_lo_z', 
            'proximity_x','proximity_y','proximity_z','proximity_mode_x','proximity_mode_y','proximity_mode_z','segments_x','segments_y','segments_z','tag_mode','np_format_id','np_table_id','size']

node_df[node_ints] = node_df[node_ints].astype(int)
node_df['record_id'] = node_df['record_id'].astype(np.int64)
node_df.to_csv("C:\\Users\\Jarrod\\Desktop\\gaia_2024-02-11_app\\User\\Prototypes\\2024-04-10\\proto-20240410T193506\\csv\\proto-20240410T193506_np_node.csv",mode='w', index=False)

tag_df[["np_tag_id", "record_id"]] = tag_df[["np_tag_id", "record_id"]].astype(int)
tag_df.to_csv("C:\\Users\\Jarrod\\Desktop\\gaia_2024-02-11_app\\User\\Prototypes\\2024-04-10\\proto-20240410T193506\\csv\\proto-20240410T193506_np_tag.csv",mode='w', index=False)